In [1]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [14]:
node_ = np.loadtxt('VRP/test_data/vrp20_test_data.csv', dtype=float, delimiter=',')
# demand_=np.loadtxt('VRP/test_data/vrp20_demand.csv', dtype=float, delimiter=',')
demand_=np.loadtxt('VRP/test_data/demand_10.csv', dtype=float, delimiter=';')
capcity_=np.loadtxt('VRP/test_data/vrp20_capcity.csv', dtype=float, delimiter=',')

In [3]:
node_ = node_[:11]
demand_= demand_[:1]
capcity_ =capcity_[:1]


In [15]:
node_

array([[0.302275, 0.826449],
       [0.202417, 0.8687  ],
       [0.925818, 0.708371],
       ...,
       [0.159775, 0.569829],
       [0.071163, 0.55179 ],
       [0.374352, 0.226966]])

In [4]:
demand_

array([[0. , 0.5, 0.5, 0.8, 0.9, 0.8, 0.9, 0.5, 0.9, 0.3, 0.2]])

In [5]:
# Ваши точки
points = np.array([[[0.302275, 0.826449],
                   [0.202417, 0.8687  ],
                   [0.925818, 0.708371],
                   [0.022478, 0.356485],
                   [0.885683, 0.976655],
                   [0.897849, 0.082953],
                   [0.26324 , 0.8684  ],
                   [0.408226, 0.350307],
                   [0.810067, 0.685676],
                   [0.64122 , 0.586787],
                   [0.993538, 0.530256]]])

# Получите количество точек
num_points = len(points[0])


In [69]:
# Инициализируйте матрицу расстояний нулями
distance_matrix = np.zeros((num_points, num_points))

# Заполните матрицу расстояний
for i in range(num_points):
    for j in range(num_points):
        # Вычислите Евклидово расстояние между точками i и j
        distance = np.linalg.norm(points[0, i] - points[0, j])
        distance_matrix[i, j] = distance
# distance_matrix = np.round(distance_matrix * 1000).astype(int)

# Преобразуйте матрицу в список списков
distance_matrix_list = distance_matrix.tolist()


In [75]:
def create_data_model():
    # Ваши точки
    points = np.array([[[0.302275, 0.826449],
                    [0.202417, 0.8687  ],
                    [0.925818, 0.708371],
                    [0.022478, 0.356485],
                    [0.885683, 0.976655],
                    [0.897849, 0.082953],
                    [0.26324 , 0.8684  ],
                    [0.408226, 0.350307],
                    [0.810067, 0.685676],
                    [0.64122 , 0.586787],
                    [0.993538, 0.530256]]])

    # Получите количество точек
    num_points = len(points[0])

    # Инициализируйте матрицу расстояний нулями
    distance_matrix = np.zeros((num_points, num_points))

    # Заполните матрицу расстояний
    for i in range(num_points):
        for j in range(num_points):
            # Вычислите Евклидово расстояние между точками i и j
            distance = np.linalg.norm(points[0, i] - points[0, j])
            distance_matrix[i, j] = distance

    # distance_matrix = np.round(distance_matrix * 1000).astype(int)

    # Преобразуйте матрицу в список списков
    distance_matrix_list = distance_matrix.tolist()

    
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix_list
    data["demands"] = [0. , 0.5, 0.5, 0.8, 0.9, 0.8, 0.9, 0.5, 0.9, 0.3, 0.2]
    data["vehicle_capacities"] = [3, 3, 3]
    data["num_vehicles"] = 3
    data["depot"] = 0
    return data

In [76]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()

Objective: 0
Route for vehicle 0:
 0 -> 0
Distance of the route: 0m

Route for vehicle 1:
 0 -> 0
Distance of the route: 0m

Route for vehicle 2:
 0 ->  10 ->  9 ->  8 ->  7 ->  6 ->  5 ->  4 ->  3 ->  2 ->  1 -> 0
Distance of the route: 0m

Maximum of the route distances: 0m


In [48]:
def create_data_model():
    # Ваши точки
    points = np.array([[[0.302275, 0.826449],
                    [0.202417, 0.8687  ],
                    [0.925818, 0.708371],
                    [0.022478, 0.356485],
                    [0.885683, 0.976655],
                    [0.897849, 0.082953],
                    [0.26324 , 0.8684  ],
                    [0.408226, 0.350307],
                    [0.810067, 0.685676],
                    [0.64122 , 0.586787],
                    [0.993538, 0.530256]]])

    # Получите количество точек
    num_points = len(points[0])

    # Инициализируйте матрицу расстояний нулями
    distance_matrix = np.zeros((num_points, num_points))

    # Заполните матрицу расстояний
    for i in range(num_points):
        for j in range(num_points):
            # Вычислите Евклидово расстояние между точками i и j
            distance = np.linalg.norm(points[0, i] - points[0, j])
            distance_matrix[i, j] = distance

    rounded_distance_matrix = np.round(distance_matrix * 1000).astype(int)
    # Преобразуйте матрицу в список списков
    distance_matrix_list = rounded_distance_matrix.tolist()

    
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix_list
    data["demands"] = [0. , 0.5, 0.5, 0.8, 0.9, 0.8, 0.9, 0.5, 0.9, 0.3, 0.2]
    data["vehicle_capacities"] = [3, 3, 3]
    data["num_vehicles"] = 3
    data["depot"] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 0
Route for vehicle 0:
 0 Load(0.0) ->  0 Load(0.0)
Distance of the route: 0m
Load of the route: 0.0

Route for vehicle 1:
 0 Load(0.0) ->  0 Load(0.0)
Distance of the route: 0m
Load of the route: 0.0

Route for vehicle 2:
 0 Load(0.0) ->  10 Load(0.2) ->  9 Load(0.5) ->  8 Load(1.4) ->  7 Load(1.9) ->  6 Load(2.8) ->  5 Load(3.5999999999999996) ->  4 Load(4.5) ->  3 Load(5.3) ->  2 Load(5.8) ->  1 Load(6.3) ->  0 Load(6.3)
Distance of the route: 0m
Load of the route: 6.3

Total distance of all routes: 0m
Total load of all routes: 6.3
